In [131]:
import os
import sys
import scipy
import importlib
import numpy as np
import pandas as pd
import networkx as nx

import torch.nn.functional as F

%matplotlib


Using matplotlib backend: MacOSX


In [ ]:
# Local imports
import GCN.model_config as conf
import GCN.model_constants as cons

In [89]:
import GCN.layers as model
import GCN.functions as fun

importlib.reload(fun)
importlib.reload(model)


<module 'GCN.layers' from '/Users/andreaparra/Dropbox/4_Work/DataLamaCovid/projects/network_forcasting/GCN/layers.py'>

## Data

In [100]:
times = [1, 2, 3, 4]
graphs = []             # Ordered timeseries
adj_matrices = []       # Ordered timeseries
node_attrs = []         # Ordered timeseries

for t in times:
    edge_data = os.path.join(conf.raw_dir, str(t), "edgelist.csv")
    node_data = os.path.join(conf.raw_dir, str(t), "node_attrs.csv")
    
    # Get node attrs
    node_attrs.append(pd.read_csv(node_data)["cases"].values)

    # load graph
    df_edge_data = pd.read_csv(edge_data)
    Graph = nx.Graph()
    G = nx.from_pandas_edgelist(df_edge_data, edge_attr='weight', create_using=Graph)
    graphs.append(G)

    # Calculate spectral normalized matrix
    A_norm = fun.calculate_spectral_norm_matrix(G)
    adj_matrices.append(A_norm)

print(f"Loaded {len(times)} graphs.")
print(f"Found {len(node_attrs[0])} nodes.")
print(f"Found {1} node attributes.")    

Loaded 4 graphs.
Found 25 nodes.
Found 1 node attributes.


## First pass simulation

In [134]:
embedding_window = 2

In [135]:
# X is the embedding of node features per a window
x = torch.tensor([node_attrs[0], node_attrs[1]], dtype=torch.float)
x = torch.transpose(x, 0, 1)
x.size()

torch.Size([25, 2])

In [136]:
# MLP layer
mlp1 = torch.nn.Linear(embedding_window, 1, bias=False)
                  
H0 = F.relu(mlp1(x))
print(f"Output of first embedding layer: {H0.size()}.")

Output of first embedding layer: torch.Size([25, 1]).


In [137]:
# Adjacency matrix
adj = torch.randn(25, 25)

# First graph layer
gc1 = model.GCN(1, 1)
H1 = gc1(H0, adj)
print(f"Output of first graph layer: {H1.size()}.")

# Concatenat H0
H1 = torch.cat([H1 , H0], dim=-1)
print(f"Output of first graph layer after adding H0: {H1.size()}.")

Output of first graph layer: torch.Size([25, 1]).
Output of first graph layer after adding H0: torch.Size([25, 2]).


In [138]:
# Second graph layer
gc2 = model.GCN(2, 1)       # the number of features is 2 because of H0
H2 = gc2(H1, adj)
print(f"Output of second graph layer: {H2.size()}.")

# Concatenat H0
H2 = torch.cat([H2 , H0], dim=-1)
print(f"Output of second graph layer after adding H0: {H2.size()}.")

Output of second graph layer: torch.Size([25, 1]).
Output of second graph layer after adding H0: torch.Size([25, 2]).


In [139]:
# Predictive layer
pred = torch.nn.Linear(2, 1, bias=False)
P = F.relu(pred(H2))
print(f"Output of predictive layer: {P.size()}.")

Output of predictive layer: torch.Size([25, 1]).


In [140]:

P

tensor([[18.4782],
        [ 0.0000],
        [ 3.5651],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 5.9865],
        [ 0.1467],
        [ 0.0000],
        [ 0.0000],
        [ 8.2315],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 9.0863],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]], grad_fn=<ReluBackward0>)